# Reinforcement Learning


## pseudo code algorithm
- Initialize G randomly
- Repeat epoch
   - while game
     - Get state / reward from env
     - select action
     - get updated state / reward
      - store new state / reward
- Replay memory of prev. epoch

### Addendum

There are different serialization formats:
- HDF5
- Pickle
- Apache:
  - Feather (arrow)
  - Parquet
  
The file formats are depend on speed and compatibility. I would rather speed at this point. I need to optimize for a single system (linux) - Pickle would be the choice.

In [1]:

import logging
import sys
import jdc  
import numpy as np
import pandas as pd

logging.basicConfig(
    filename='mylog.log',
    level=logging.DEBUG,
    format=' %(asctime)s - %(levelname)s - %(message)s',

    )
logger = logging.getLogger()
logger.addHandler(logging.StreamHandler(stream=sys.stdout))


## Agent

interact with environment 
does not choose reward
has to have: 
- learning 
- choose action
- updating memory 


We will set learning rate alpha \
G is the Q table \
we have to initialize the q table 

In [3]:
direction ={
    'U' : [1, -1, 0],
    'R': [2, 0, 1],
    'D': [3 , 1 , 0],
    'L' : [4 , 0 , -1]
    }
class Agent (object):
    def __init__(self, states, alpha=0.15, randomFactor=0.2) -> None:
        self.state_history = None
        self.alpha = 0.1
        self.G = {}
        self.initReward(states)
        

### learning



>  The equation used to update the rewards is a derivative of the SMA (bellman's equation):
$$ SMA = {\Sigma _ {n}  a _ {1} + a _ {2} + \ldots  a _ {n} \over n} = {{\hat {x}_{n-1}} + { 1\over n}( x_n-\hat {x}_{n-1} )} $$
$$ G_{state} = G_{state} + \alpha (\mathfrak{target}-G_{state}) $$

if alpha is: 
- 1 = target
$$ G_{state} = G_{state} + \mathfrak{target} -G_{state} = \mathfrak{target} $$

- 0 = current
$$ G_{state} = G_{state} $$






In [5]:
%%add_to Agent
def learning (self) -> None:
    target = 0 # after epoche
    for prev, reward in reversed(self.stateHistory):
        self.G[prev] = self.G + self.alpha* (target - self.G[prev])
        target += reward
        self.stateHistory[None]

### choose_action
- action space
  - dictionary of actions
- choose action
  - look at G state - reward
- update state history
  - addstate to history
* forgot to do exploration v exploitation

In [6]:
%%add_to Agent
def choose_action(self, states, allowedMoves) -> int:

        reward = 0
        nextState = (None, None)
        robot = np.nonzero(env == 2) #get robot location
        randomN = np.random.random()
        if randomN > self.randomFactor:
                for a, x, y in allowedMoves.values(): #pick best action | reward
                        nextState[0] = robot[0]+x 
                        nextState[1] = robot[1]+y
                        if reward < self.G[nextState]: # get reward
                                choice = nextState
        else: 
                nextMove = np.random.choice(allowedMoves)


        self.stateHistory.append (choice) # add to state history
        return choice

### mem_update

In [7]:
%%add_to Agent
def mem_update (self) -> None:
        pass

The Q table is initialized here with a random distribution between 0 and 1:

In [8]:
%%add_to Agent
def initReward (self, states) -> None:
        for state in states:
                self.G[state] = np.random.uniform(low=-1.0, high=-0.1)
        pass

## Environment
- maze definition
- update maze
- allowed moves
- check or game over
- get reward
- print maze

Anything that is not the agent. 
- reward
- observation
- states 
<br>
Robot can not move a wall (robot )
check if game is over
state of system
get reward
print maze


### My implementation
#### We re going to design a maze:
- 6x6
- 0,0 entry
- 5,5 exit




##### maze
* numpy array:
  - 0 empty
  - 1 wall
  - 2 robot

import numpy as np
import pandas as pd
maze = np.zeros([6,6], int)

def viz_maze(array)-> None:
    print (pd.DataFrame(array))

def reset_env(env):
    env = np.zeros([6,6], int)
    env[0,0] = 2 #robot
    return env
    

maze = reset_env(maze)
viz_maze(maze)


###### Add Walls

maze = reset_env(maze)

def add_walls(env):
    for i in range (4):
        env [i,5] = 1
    for i in range (5):
        env [5,i] = 1
    for i in range(2,5):
        env[2,i] = 1
    env[3,2] = 1
    return maze

        
maze = add_walls(maze)
viz_maze(maze)


#### Allow move

Allow move to zero slots <br>
if wall disable move 

- https://medium.com/@cy.chiang/python-if-else-statement-vs-dictionary-c156b226c04d
- https://github.com/cychiang/python-for-fun/blob/master/dict_ifelse/app.py

def location (env) -> list:
    agt = np.nonzero(env == 2)
    wall = np.nonzero(env == 1)
    return agt, wall

location(maze)

timesExec = 0

def move(dir)->bool:
    global timesExec 
    timesExec += 1
    direction ={
        'up' : [1,-1,0],
        'right': [2, 0, 1],
        'down': [3 , 1 ,0],
        'left' : [4 , 0 ,-1]
    }

    if type(dir) != int:
        try:
            dir = direction[dir][0] #get int of move
        except KeyError:
            logger.critical("ops: {} not support".format(dir))
    else:
        if dir > 4:
            raise ValueError("out of movement range")
        dir = dir
    
    xdir = 0
    ydir = 0
    result = dir

    for new_k in direction.items():
        if new_k[1][0] == dir:
            result = new_k[0]
            xdir = new_k[1][1]
            ydir = new_k[1][2]


    print(result, xdir, ydir)

    
    agt, wall = location(maze)

    nagt = [None, None]

    nagt[0] = agt[0]+xdir if  0 <= agt[0]+xdir <= 5 else agt[0] # boundary check
    nagt[1] = agt[1]+ydir if 0 <= agt[1]+ydir <= 5 else agt[1]

    wall = np.transpose(wall)

    for x,y in wall:
        if nagt[0] == x and nagt[1] == y:
            nagt = agt
            print ('hit wall')

    if agt[0] == nagt[0] and agt[1] == nagt [1]:
        print ('no movement')
        return 0
    else:
        maze[agt[0], agt[1]] = 0
        maze[nagt[0], nagt[1]] = 2
        return 1

move (4)
viz_maze(maze)

#### Reward

points = 0 
gamma = 0.4
def reward(reward, moves, gamma):
    reward = reward+gamma**(1/moves)
    print (reward)



if move('left'):    
    reward (points, timesExec, gamma)
viz_maze(maze)

### School

#### maze definition

In [9]:


class Maze (object):
    def __init__(self) -> None:
        self.reset_env()


    def reset_env(self)->None:
        self.maze = np.zeros([6,6], int)
        self.maze [:4,5] = 1
        self.maze [5,:5] = 1
        self.maze[2,2:] = 1
        self.maze[3,2] = 1
        self.maze[0,0] = 2 #robot
        self.robotPostition = (0,0)


##### print maze

In [10]:
%%add_to Maze

def printMaze(self)-> None:
    print (pd.DataFrame(self.maze))



##### Allowed Moves

In [11]:
%%add_to Maze

def isAloowedMove (self, state, action)-> None:
    pass

##### update maze

In [12]:
%%add_to Maze

def updateMaze (self)-> None:
    pass
    

##### check for game over

In [13]:
%%add_to Maze

def isGameOver (self)-> None:
    pass

##### Get State

In [14]:
%%add_to Maze

def getState (self)-> None:
    pass

##### get reward
- dont over specify 
- randomly assign reward

In [15]:
%%add_to Maze

def giveReward (self)-> None:
    pass

#### running

In [17]:
env = Maze()
env.printMaze()

   0  1  2  3  4  5
0  2  0  0  0  0  1
1  0  0  0  0  0  1
2  0  0  1  1  1  1
3  0  0  1  0  0  1
4  0  0  0  0  0  0
5  1  1  1  1  1  0
